# us-east-2

In [18]:
# specify the aws profile and region you want to use to deploy the stack
aws_profile="bbdev"
region_name="us-east-2"

MEGA = 1024*1024

In [19]:
lambda_name = !docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/mnt/state_vol \
    cloudfuse/cloud-reader-terraform output lambda_arn
lambda_name = lambda_name[0][1:len(lambda_name[0])-1]
print('lambda_name:', lambda_name)

lambda_name: arn:aws:lambda:us-east-2:615900053518:function:cloud-reader-benchmark


In [21]:
import boto3
import json
import base64

session = boto3.Session(profile_name=aws_profile)
client = session.client('lambda', region_name = region_name)

In [31]:
def bench_setup_1(max_parallel, ranges):
    inputParams = {
        "region": region_name,
        "bucket": "cloudfuse-taxi-data",
        "key": "tpch/tbl-s1/lineitem.tbl",
        "size": 759863287,
        "ranges": ranges,
        "max_parallel": max_parallel,
    }
    response = client.invoke(
        FunctionName = lambda_name,
        InvocationType = 'RequestResponse',
        Payload = json.dumps(inputParams),
        LogType='Tail'
    )
    print(base64.b64decode(response['LogResult']).decode("utf-8") )
    print(json.load(response['Payload']))


### 100MB chunks

In [32]:
bench_setup_1(32, [{"start": 0, "length": 100*MEGA}])

START RequestId: c2de5630-3de9-4b7e-838b-41ed68a9e933 Version: $LATEST
END RequestId: c2de5630-3de9-4b7e-838b-41ed68a9e933
REPORT RequestId: c2de5630-3de9-4b7e-838b-41ed68a9e933	Duration: 1285.74 ms	Billed Duration: 1321 ms	Memory Size: 3008 MB	Max Memory Used: 132 MB	Init Duration: 34.26 ms	

{'init_duration': 18, 'range_durations': [1262]}


In [33]:
bench_setup_1(32, [{"start": 0, "length": 100*MEGA}])

START RequestId: 323d3c03-1208-465b-9aa6-e98999b4f5ac Version: $LATEST
END RequestId: 323d3c03-1208-465b-9aa6-e98999b4f5ac
REPORT RequestId: 323d3c03-1208-465b-9aa6-e98999b4f5ac	Duration: 1116.33 ms	Billed Duration: 1117 ms	Memory Size: 3008 MB	Max Memory Used: 232 MB	

{'init_duration': 0, 'range_durations': [1115]}


### 10MB chunks

In [34]:
bench_setup_1(32, [{"start": 0, "length": 10*MEGA}])

START RequestId: 280fc413-e624-4c02-aded-4912dd0d60bb Version: $LATEST
END RequestId: 280fc413-e624-4c02-aded-4912dd0d60bb
REPORT RequestId: 280fc413-e624-4c02-aded-4912dd0d60bb	Duration: 112.30 ms	Billed Duration: 113 ms	Memory Size: 3008 MB	Max Memory Used: 242 MB	

{'init_duration': 0, 'range_durations': [111]}


In [35]:
bench_setup_1(32, [{"start": 0, "length": 10*MEGA}])

START RequestId: c6b360cf-4f1e-4a67-b0db-39ddd8b67a29 Version: $LATEST
END RequestId: c6b360cf-4f1e-4a67-b0db-39ddd8b67a29
REPORT RequestId: c6b360cf-4f1e-4a67-b0db-39ddd8b67a29	Duration: 108.39 ms	Billed Duration: 109 ms	Memory Size: 3008 MB	Max Memory Used: 252 MB	

{'init_duration': 0, 'range_durations': [107]}


In [86]:
bench_setup_1(32, [{"start": i*10, "length": 10*MEGA} for i in range(0,10)])

START RequestId: bab974b3-6a2e-4678-97fc-24cd46f6e77a Version: $LATEST
END RequestId: bab974b3-6a2e-4678-97fc-24cd46f6e77a
REPORT RequestId: bab974b3-6a2e-4678-97fc-24cd46f6e77a	Duration: 11429.90 ms	Billed Duration: 11430 ms	Memory Size: 3008 MB	Max Memory Used: 3009 MB	
RequestId: bab974b3-6a2e-4678-97fc-24cd46f6e77a Error: Runtime exited with error: signal: killed
Runtime.ExitError

{'errorMessage': 'RequestId: bab974b3-6a2e-4678-97fc-24cd46f6e77a Error: Runtime exited with error: signal: killed', 'errorType': 'Runtime.ExitError'}


In [45]:
bench_setup_1(32, [{"start": i*10, "length": 10*MEGA} for i in range(0,20)])

START RequestId: 78d20f32-a642-4eeb-bf18-65e3f228174c Version: $LATEST
END RequestId: 78d20f32-a642-4eeb-bf18-65e3f228174c
REPORT RequestId: 78d20f32-a642-4eeb-bf18-65e3f228174c	Duration: 948.63 ms	Billed Duration: 949 ms	Memory Size: 3008 MB	Max Memory Used: 1875 MB	

{'init_duration': 0, 'range_durations': [786, 786, 929, 929, 929, 929, 929, 929, 929, 945, 945, 946, 946, 946, 947, 947, 947, 947, 947, 947]}


In [46]:
bench_setup_1(10, [{"start": i*10, "length": 10*MEGA} for i in range(0,20)])

START RequestId: d13ce5ca-b613-4f8a-ba69-2f2b9065cd48 Version: $LATEST
END RequestId: d13ce5ca-b613-4f8a-ba69-2f2b9065cd48
REPORT RequestId: d13ce5ca-b613-4f8a-ba69-2f2b9065cd48	Duration: 1973.40 ms	Billed Duration: 1974 ms	Memory Size: 3008 MB	Max Memory Used: 2072 MB	

{'init_duration': 0, 'range_durations': [1519, 1519, 1519, 1519, 1519, 1519, 1519, 1519, 1806, 1806, 1806, 1806, 1806, 1806, 1806, 1836, 1836, 1972, 1972, 1972]}


### Notes:
- *Max Memory Used* keeps increasing execution after execution (until OOM)
- check that downloading 20 chunks is only 2x the duration of 10 chunks (with the proper `max_parallel`)

# eu-west-1

In [12]:
# specify the aws profile and region you want to use to deploy the stack
aws_profile="bbdev"
region_name="eu-west-1"

MEGA = 1024*1024

In [13]:
lambda_name = !docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/mnt/state_vol \
    cloudfuse/cloud-reader-terraform output lambda_arn
lambda_name = lambda_name[0][1:len(lambda_name[0])-1]
print('lambda_name:', lambda_name)

lambda_name: arn:aws:lambda:eu-west-1:615900053518:function:cloud-reader-benchmark


In [14]:
import boto3
import json
import base64

session = boto3.Session(profile_name=aws_profile)
client = session.client('lambda', region_name = region_name)

In [15]:
def bench_setup_1(ranges):
    inputParams = {
        "region": region_name,
        "bucket": "cloudfuse-taxi-data-eu",
        "key": "tpch/tbl-s1/lineitem.tbl",
        "size": 759863287,
        "ranges": ranges
    }
    response = client.invoke(
        FunctionName = lambda_name,
        InvocationType = 'RequestResponse',
        Payload = json.dumps(inputParams),
        LogType='Tail'
    )
    print(base64.b64decode(response['LogResult']).decode("utf-8") )
    print(json.load(response['Payload']))


In [16]:
bench_setup_1([{"start": 0, "length": 100*MEGA}])

START RequestId: 303a4bb6-bff4-46e0-8b98-7c2112e5bf0d Version: $LATEST
END RequestId: 303a4bb6-bff4-46e0-8b98-7c2112e5bf0d
REPORT RequestId: 303a4bb6-bff4-46e0-8b98-7c2112e5bf0d	Duration: 1098.28 ms	Billed Duration: 1134 ms	Memory Size: 3008 MB	Max Memory Used: 132 MB	Init Duration: 34.99 ms	

{'init_duration': 20, 'range_durations': [1072]}
